# Some Experiments
used for some shit

## Imports

In [29]:
import sys
import os

# Füge das übergeordnete Verzeichnis zu sys.path hinzu
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

In [30]:
from datetime import datetime
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from copy import deepcopy as dc

# own classes
from utilities import save_unscaled_sequential_data, load_sequential_time_series, train_test_split, extract_features_and_targets_reg, extract_features_and_targets, split_data_into_sequences, Scaler, ValidationLossAccumulationCallback, accuracy
from baseline_model.LSTM import LSTMRegression, train_model, LSTMClassification
from baseline_model.TimeSeriesDataset import TimeSeriesDataset
from baseline_model.mean_regressor import MeanRegressor
from data_generation.algorithmic.brownian_motion import brownian_motion
from data_generation.algorithmic.jittering import create_jittered_time_series

In [31]:
DATA_FOLDER = Path("./data")
SYNTHETIC_DATA_FOLDER = DATA_FOLDER / "synthetic"
REAL_DATA_FOLDER = DATA_FOLDER / "real"
LOAD_ORIGINAL_DATA = False

In [32]:
traffic_le = pd.read_csv(REAL_DATA_FOLDER / "metro_interstate_traffic_volume_label_encoded.csv")

## Jittering

In [33]:
jitter_factor = 0.2
jitt, noise = create_jittered_time_series(traffic_le, jitter_factor=jitter_factor, no_features=5)

In [34]:
jitt.to_csv(SYNTHETIC_DATA_FOLDER / f'mitv_jittered_{str(jitter_factor).replace(".", "")}.csv', index=False)

In [35]:
jitt.head()

,traffic_volume,temp,rain_1h,snow_1h,clouds_all,month,hour,weather_main,weather_description,weekend,holiday
0,0.000000,291.796010,0.255222,0.000000,0.000000,7.0,1.0,0.0,24.0,0.0,0.0
1,369.261790,285.281771,0.000000,0.000000,12.825643,7.0,2.0,0.0,24.0,0.0,0.0
2,704.568027,290.945099,0.185225,0.001483,0.000000,7.0,3.0,6.0,12.0,0.0,0.0
3,1369.312394,286.628933,0.025249,0.000832,0.000000,7.0,4.0,0.0,24.0,0.0,0.0
4,2951.423930,285.980158,0.000000,0.000000,5.257887,7.0,5.0,0.0,24.0,0.0,0.0


In [36]:
traffic_le.head()

,traffic_volume,temp,rain_1h,snow_1h,clouds_all,month,hour,weather_main,weather_description,weekend,holiday
0,444,289.40,0.0,0.0,1,7,1,0,24,0,0
1,347,289.19,0.0,0.0,1,7,2,0,24,0,0
2,356,288.74,0.3,0.0,1,7,3,6,12,0,0
3,936,288.07,0.0,0.0,1,7,4,0,24,0,0
4,2886,287.87,0.0,0.0,1,7,5,0,24,0,0


In [37]:
jitt.describe()

,traffic_volume,temp,rain_1h,snow_1h,clouds_all,month,hour,weather_main,weather_description,weekend,holiday
count,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000
mean,3316.383778,282.704151,0.114660,0.000906,43.199934,6.635509,11.500403,2.211006,16.516537,0.286205,0.001298
std,2001.155424,12.601459,0.679160,0.008343,38.805027,3.343684,6.922094,2.719176,8.206819,0.451994,0.036001
min,0.000000,241.275376,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1365.194108,273.436620,0.000000,0.000000,3.163873,4.000000,6.000000,0.000000,12.000000,0.000000,0.000000
50%,3525.105311,284.344137,0.008047,0.000000,36.468100,7.000000,12.000000,1.000000,18.000000,0.000000,0.000000
75%,4981.868520,292.839681,0.104600,0.001114,83.883017,9.000000,17.500000,5.000000,24.000000,1.000000,0.000000
max,7971.060457,313.631501,41.994010,0.511349,118.784412,12.000000,23.000000,9.000000,33.000000,1.000000,1.000000


In [38]:
traffic_le.describe()

,traffic_volume,temp,rain_1h,snow_1h,clouds_all,month,hour,weather_main,weather_description,weekend,holiday
count,28511.00000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000,28511.000000
mean,3313.74238,282.688768,0.061611,0.000250,42.122795,6.635509,11.500403,2.211006,16.516537,0.286205,0.001298
std,1971.53206,12.367361,0.678185,0.008298,39.316195,3.343684,6.922094,2.719176,8.206819,0.451994,0.036001
min,0.00000,243.390000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1289.00000,273.480000,0.000000,0.000000,1.000000,4.000000,6.000000,0.000000,12.000000,0.000000,0.000000
50%,3507.00000,284.550000,0.000000,0.000000,40.000000,7.000000,12.000000,1.000000,18.000000,0.000000,0.000000
75%,4948.00000,292.790000,0.000000,0.000000,90.000000,9.000000,17.500000,5.000000,24.000000,1.000000,0.000000
max,7280.00000,310.070000,42.000000,0.510000,100.000000,12.000000,23.000000,9.000000,33.000000,1.000000,1.000000


## Rescale timeGAN data

In [39]:
# save_unscaled_sequential_data(ori_data_path=REAL_DATA_FOLDER / "metro_interstate_traffic_volume_label_encoded.csv",
#                                      scaled_data_path=SYNTHETIC_DATA_FOLDER / "mitv_28499_12_11_lstm.csv",
#                                      scaled_data_shape=(28499, 12, 11),
#                                      no_features_to_inverse_scale=9)